### Pyspark
#### Linear Regression

In [0]:
#Import the libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [0]:
#Load and read dataset
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/praneethsanthosh555@gmail.com/Admission_Prediction.csv",header=True)

In [0]:
#Show the dataset
df1.show(5)

+---------+-----------+-----------------+----+----+----+--------+---------------+
GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
 337.00| 118.00| 4|4.50|4.50|9.65| 1.00| 0.92|
 324.00| 107.00| 4|4.00|4.50|8.87| 1.00| 0.76|
 null| 104.00| 3|3.00|3.50|8.00| 1.00| 0.72|
 322.00| 110.00| 3|3.50|2.50|8.67| 1.00| 0.80|
 314.00| 103.00| 2|2.00|3.00|8.21| 0.00| 0.65|
+---------+-----------+-----------------+----+----+----+--------+---------------+
only showing top 5 rows

In [0]:
df1.printSchema()

root
-- GRE Score: string (nullable = true)
-- TOEFL Score: string (nullable = true)
-- University Rating: string (nullable = true)
-- SOP: string (nullable = true)
-- LOR: string (nullable = true)
-- CGPA: string (nullable = true)
-- Research: string (nullable = true)
-- Chance of Admit: string (nullable = true)

In [0]:
#From sql function gettig all the columns whith the help of col obejct
from pyspark.sql.functions import col
new_data=df1.select(*(col(c).cast('float').alias(c) for c in df1.columns))

In [0]:
#Printschema to get the info of the data type of the each variable and null values count (Bollian True or False)
new_data.printSchema()

root
-- GRE Score: float (nullable = true)
-- TOEFL Score: float (nullable = true)
-- University Rating: float (nullable = true)
-- SOP: float (nullable = true)
-- LOR: float (nullable = true)
-- CGPA: float (nullable = true)
-- Research: float (nullable = true)
-- Chance of Admit: float (nullable = true)

In [0]:
#Check for null values are present in the data
from pyspark.sql.functions import col, when , count, isnan
new_data.select([count(when(col(c).isNull(),c)).alias(c) for c in df1.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
 15| 10| 15| 0| 0| 0| 0| 0|
+---------+-----------+-----------------+---+---+----+--------+---------------+

In [0]:
#impute the null values 
from pyspark.ml.feature import Imputer
imputer=Imputer(inputCols=["GRE Score","TOEFL Score","University Rating"],
                       outputCols=["GRE Score","TOEFL Score","University Rating"])
model=imputer.fit(new_data)

imputed_data=model.transform(new_data)

In [0]:
imputed_data.select([count(when(col(c).isNull(),c)).alias(c) for c in new_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0|
+---------+-----------+-----------------+---+---+----+--------+---------------+

In [0]:
imputed_data.show(5)

+---------+-----------+-----------------+---+---+----+--------+---------------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
 337.0| 118.0| 4.0|4.5|4.5|9.65| 1.0| 0.92|
 324.0| 107.0| 4.0|4.0|4.5|8.87| 1.0| 0.76|
316.55878| 104.0| 3.0|3.0|3.5| 8.0| 1.0| 0.72|
 322.0| 110.0| 3.0|3.5|2.5|8.67| 1.0| 0.8|
 314.0| 103.0| 2.0|2.0|3.0|8.21| 0.0| 0.65|
+---------+-----------+-----------------+---+---+----+--------+---------------+
only showing top 5 rows

In [0]:
#Split the data in to x and y
features = imputed_data.drop('Chance of Admit')
features.show(5)

+---------+-----------+-----------------+---+---+----+--------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|
+---------+-----------+-----------------+---+---+----+--------+
 337.0| 118.0| 4.0|4.5|4.5|9.65| 1.0|
 324.0| 107.0| 4.0|4.0|4.5|8.87| 1.0|
316.55878| 104.0| 3.0|3.0|3.5| 8.0| 1.0|
 322.0| 110.0| 3.0|3.5|2.5|8.67| 1.0|
 314.0| 103.0| 2.0|2.0|3.0|8.21| 0.0|
+---------+-----------+-----------------+---+---+----+--------+
only showing top 5 rows

In [0]:
#Let's assemble all the columns are together
assemble= VectorAssembler(inputCols=features.columns,
                         outputCol="features")

In [0]:
output=assemble.transform(imputed_data)

In [0]:
output=output.select("features","Chance of Admit")
output.show(5)

+--------------------+---------------+
 features|Chance of Admit|
+--------------------+---------------+
[337.0,118.0,4.0,...| 0.92|
[324.0,107.0,4.0,...| 0.76|
[316.558776855468...| 0.72|
[322.0,110.0,3.0,...| 0.8|
[314.0,103.0,2.0,...| 0.65|
+--------------------+---------------+
only showing top 5 rows

In [0]:
train_df,test_df=output.randomSplit([0.7,0.3])

In [0]:
train_df.show(5)
test_df.show(5)

+--------------------+---------------+
 features|Chance of Admit|
+--------------------+---------------+
[290.0,104.0,4.0,...| 0.45|
[294.0,93.0,1.0,1...| 0.46|
[295.0,93.0,1.0,2...| 0.46|
[295.0,96.0,2.0,1...| 0.47|
[295.0,99.0,1.0,2...| 0.37|
+--------------------+---------------+
only showing top 5 rows

+--------------------+---------------+
 features|Chance of Admit|
+--------------------+---------------+
[290.0,100.0,1.0,...| 0.47|
[293.0,97.0,2.0,2...| 0.64|
[294.0,95.0,1.0,1...| 0.49|
[295.0,99.0,2.0,2...| 0.57|
[296.0,97.0,2.0,1...| 0.49|
+--------------------+---------------+
only showing top 5 rows

In [0]:
linear_reg=LinearRegression(featuresCol="features",labelCol="Chance of Admit")
linear_model=linear_reg.fit(train_df)

In [0]:
print("Coefficients: ",linear_model.coefficients[0])
print("Intercept: ",linear_model.intercept)

Coefficients: 0.0016884199917549998
Intercept: -1.2226103674198092

In [0]:
train_summary=linear_model.summary
print("RMSE: %f" % train_summary.rootMeanSquaredError)
print('R2:  %f' % train_summary.r2)
print("MAE: %f " % train_summary.meanAbsoluteError)

RMSE: 0.056975
R2: 0.826373
MAE: 0.041186

In [0]:
#predictions
prediction=linear_model.transform(test_df)
prediction.select("prediction","Chance of Admit","features").show()

+-------------------+---------------+--------------------+
 prediction|Chance of Admit| features|
+-------------------+---------------+--------------------+
 0.4835771714325814| 0.47|[290.0,100.0,1.0,...|
 0.579312332649824| 0.64|[293.0,97.0,2.0,2...|
 0.4792021134413895| 0.49|[294.0,95.0,1.0,1...|
 0.522553744076059| 0.57|[295.0,99.0,2.0,2...|
 0.5185173435945782| 0.49|[296.0,97.0,2.0,1...|
 0.5227331351343294| 0.43|[297.0,96.0,2.0,2...|
 0.5610116888977366| 0.54|[297.0,99.0,4.0,3...|
 0.5751559290533401| 0.34|[298.0,98.0,2.0,4...|
 0.617830416130619| 0.58|[298.0,100.0,3.0,...|
 0.5284087082043629| 0.54|[299.0,96.0,2.0,1...|
 0.5391530732905667| 0.59|[299.0,100.0,1.0,...|
 0.5622241488349049| 0.63|[299.0,100.0,3.0,...|
 0.5054840802536205| 0.42|[299.0,100.0,3.0,...|
 0.670430713716812| 0.56|[299.0,102.0,3.0,...|
 0.5612414174672427| 0.61|[300.0,98.0,1.0,2...|
0.43982203504694684| 0.36|[300.0,99.0,1.0,3...|
 0.690222537513111| 0.64|[300.0,100.0,3.0,...|
 0.6210307492361336| 0.62|[300.0,100.0,3.12...|
 0.6113391459391742| 0.71|[300.0,104.0,3.0,...|
 0.5289980336434388| 0.54|[301.0,96.0,1.0,3...|
+-------------------+---------------+--------------------+
only showing top 20 rows

In [0]:
#Get the prediction score
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluater=RegressionEvaluator(predictionCol="prediction",
                                  labelCol="Chance of Admit",
                                  metricName="r2")
print("R2 error (r2) is found to be %g" % pred_evaluater.evaluate(prediction))

R2 error (r2) is found to be 0.807215